In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import genpareto
matplotlib.use('TkAgg')

from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from tqdm.notebook import trange, tqdm
#from anomaly_scoring import get_anomaly_scores

import torch
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline 


from utils import get_taxi_data_VAE, get_taxi_data_cVAE
from utils import softclip
from utils import plot_train_test_reconstructions, plot_train_test_reconstructions_prob_decoder_model
from models.cnn_sigmaVAE import CNN_sigmaVAE

from maf import MAF













import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torch.autograd import Variable

from torchsummary import summary

import torchvision
from torchvision import datasets
from torchvision import transforms

In [2]:
dataset_path = '../../datasets/nab/NAB-master/data/realKnownCause/nyc_taxi.csv'
#dataset_path = '../../datasets/nab/NAB-master/data/realKnownCause/machine_temperature_system_failure.csv'
#dataset_path = '../../datasets/nab/NAB-master/data/realKnownCause/ambient_temperature_system_failure.csv'
#dataset_path = '../../datasets/nab/NAB-master/data/realKnownCause/cpu_utilization_asg_misconfiguration.csv'
#dataset_path = '../../datasets/nab/NAB-master/data/realKnownCause/ec2_request_latency_system_failure.csv'


if 'nyc_taxi' in dataset_path:
    anomaly_timestamps = ["2014-11-01 19:00:00", "2014-11-27 15:30:00","2014-12-25 15:00:00","2015-01-01 01:00:00","2015-01-27 00:00:00"]

    anomaly_windows = [
        [
            "2014-10-30 15:30:00",
            "2014-11-03 22:30:00"
        ],
        [
            "2014-11-25 12:00:00",
            "2014-11-29 19:00:00"
        ],
        [
            "2014-12-23 11:30:00",
            "2014-12-27 18:30:00"
        ],
        [
            "2014-12-29 21:30:00",
            "2015-01-03 04:30:00"
        ],
        [
            "2015-01-24 20:30:00",
            "2015-01-29 03:30:00"
        ]
    ]
    
elif 'ambient_temperature_system_failure' in dataset_path:
    anomaly_timestamps = ["2013-12-22 20:00:00","2014-04-13 09:00:00"]

elif 'cpu_utilization_asg_misconfiguration' in dataset_path:
    anomaly_timestamps = ["2014-07-12 02:04:00","2014-07-14 21:44:00"]

elif 'ec2_request_latency_system_failure' in dataset_path:
    anomaly_timestamps = ["2014-03-14 09:06:00","2014-03-18 22:41:00","2014-03-21 03:01:00"]

elif 'machine_temperature_system_failure' in dataset_path:
    anomaly_timestamps = ["2013-12-11 06:00:00","2013-12-16 17:25:00","2014-01-28 13:55:00","2014-02-08 14:30:00"]

    

    
#singular anomaly points
'''
window_size=128
X_train_data, X_test_data, X_train_tensor, X_test_tensor, trainloader, testloader = get_taxi_data_VAE(dataset_path,window_size=window_size,train_test_split=.5) 

data = pd.read_csv(dataset_path)

timestamps = list(pd.read_csv(dataset_path)['timestamp'])
    
anomaly_idxs = []
for timestamp in anomaly_timestamps:
    start = timestamp

    loc_start = timestamps.index(start)
    anomaly_idxs.append(loc_start)
    
anomaly_idxs = np.array(anomaly_idxs)    
'''


#'''

window_size=20
X_train_data, X_test_data, X_train_tensor, X_test_tensor, trainloader, testloader = get_taxi_data_VAE(dataset_path,window_size=window_size,train_test_split=.5) 

data = pd.read_csv(dataset_path)

timestamps = list(pd.read_csv(dataset_path)['timestamp'])

anomaly_idxs = []
for timestamp in anomaly_windows:
    start, end = timestamp[0], timestamp[1]

    loc_start = timestamps.index(start)
    loc_end = timestamps.index(end)

    for i in range(loc_start, loc_end):
        anomaly_idxs.append(i)

anomaly_idxs = np.array(anomaly_idxs)
anomaly_idxs = anomaly_idxs[anomaly_idxs > 0]

plt.figure(figsize=(30,9))
for ai in anomaly_idxs:
    plt.scatter(ai, 1.7)

#'''

torch.Size([258, 1, 20]) torch.Size([258, 1, 20])
torch.Size([258, 1, 20]) torch.Size([258, 1, 20])


In [3]:

class CNN_sigmaVAE_flow(nn.Module):

    def __init__(self,latent_dim=8, window_size=20, use_probabilistic_decoder=False, flow_type = 'MAF'):
        super(CNN_sigmaVAE_flow, self).__init__()
        
        self.window_size=window_size
        self.latent_dim = latent_dim
        self.prob_decoder = use_probabilistic_decoder
        self.flow_type=flow_type
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=4, kernel_size=5, stride=1, padding=0)

        self.fc41 = nn.Linear(4*8, self.latent_dim)
        self.fc42 = nn.Linear(4*8, self.latent_dim)

        self.defc1 = nn.Linear(self.latent_dim, 4*8)
        
        self.deconv1 = nn.ConvTranspose1d(in_channels=4, out_channels=16, kernel_size=5, stride=1, padding=0, output_padding=0)
        self.deconv2 = nn.ConvTranspose1d(in_channels=16, out_channels=8, kernel_size=5, stride=1, padding=0, output_padding=0)
        self.deconv3 = nn.ConvTranspose1d(in_channels=8, out_channels=1, kernel_size=5, stride=1, padding=0, output_padding=0)

        self.decoder_fc41 = nn.Linear(window_size, window_size)
        self.decoder_fc42 = nn.Linear(window_size, window_size)
        
        self.decoder_fc43 = nn.Linear(window_size, window_size)
        self.decoder_fc44 = nn.Linear(window_size, window_size)
        
        self.log_sigma = torch.nn.Parameter(torch.full((1,), 0.0)[0], requires_grad=True)
        
        if self.flow_type=='RealNVP':
            self.flow = RealNVP(n_blocks=1, input_size=self.latent_dim, hidden_size=50, n_hidden=1)
        
        elif self.flow_type=='MAF':
            self.flow = MAF(n_blocks=1, input_size=self.latent_dim, hidden_size=50, n_hidden=1)
        
        
    def encoder(self, x):
        concat_input = x #torch.cat([x, c], 1)
        h = F.relu(self.conv1(concat_input))
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        
        self.saved_dim = [h.size(1), h.size(2)]
        
        h = h.view(h.size(0), h.size(1) * h.size(2))
        
        return self.fc41(h), self.fc42(h)
    
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add(mu) # return z sample
    
    def decoder(self, z):
        concat_input = z #torch.cat([z, c], 1)
        concat_input = self.defc1(concat_input)
        concat_input = concat_input.view(concat_input.size(0), self.saved_dim[0], self.saved_dim[1])
        
        h = F.relu(self.deconv1(concat_input))
        h = F.relu(self.deconv2(h))
        
        out = torch.sigmoid(self.deconv3(h))
        
        if self.prob_decoder:
            rec_mu = self.decoder_fc43(out).tanh()
            rec_sigma = self.decoder_fc44(out).tanh()
            return out, rec_mu, rec_sigma
        
        #else:
        return out, 0, 0
    
    
    def latent_not_planar(self, x, z_params):
        n_batch = x.size(0)
                
        # Retrieve set of parameters
        #mu, sigma = z_params
        mu, log_var = z_params
        sigma = torch.sqrt(log_var.exp())
        
        # Obtain our first set of latent points
        z0 = self.sampling(mu, log_var)
        
        zk, loss = self.flow.log_prob(z0, None)
        loss = -loss.mean(0)

        return zk, loss
            
 
    def forward(self, x):
        
        #mu, log_var = self.encoder(x)
        #z = self.sampling(mu, log_var)
        #output = self.decoder(z)
        #return output, mu, log_var
        
        z_params = self.encoder(x)
        mu, log_var = z_params
        
        z_k, kl = self.latent_not_planar(x, z_params)
        
        output, rec_mu, rec_sigma = self.decoder(z_k)
   
        return output, rec_mu, rec_sigma, kl
        
        
        
    def gaussian_nll(self, mu, log_sigma, x):
        return 0.5 * torch.pow((x - mu) / log_sigma.exp(), 2) + log_sigma + 0.5 * np.log(2 * np.pi)

    
    def reconstruction_loss(self, x_hat, x):

        log_sigma = self.log_sigma
        log_sigma = softclip(log_sigma, -6)
        
        rec_comps = self.gaussian_nll(x_hat, log_sigma, x)
        rec = rec_comps.sum()

        return rec_comps, rec

    
    def loss_function(self, recon_x, x, rec_mu, rec_sigma, kl):
        
        rec_comps, rec = self.reconstruction_loss(recon_x, x)
        #kl = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        rec_mu_sigma_loss = 0
        if self.prob_decoder:
            rec_mu_sigma_loss = self.gaussian_nll(rec_mu, rec_sigma, x).sum()
        
        return rec_comps, rec, rec_mu_sigma_loss, kl


In [4]:
def train_flow_model(model, num_epochs, learning_rate, dataloader):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    epochs=num_epochs
    tq = tqdm(range(epochs))
    for epoch in tq:
        flag = False
        for j, data in enumerate(dataloader, 0):
            
            model.train()
            optimizer.zero_grad()

            #batches
            inputs, labels = data
            inputs = inputs.cuda() if torch.cuda.is_available() else inputs.cpu()
            inputs.to(device)
            labels = labels.cuda() if torch.cuda.is_available() else labels.cpu()
            labels.to(device)

            outputs, rec_mu, rec_sigma, kl = model(inputs)

            rec_comps, rec, rec_mu_sigma_loss, kl = model.loss_function(outputs, labels, rec_mu, rec_sigma, kl)

            loss = rec + kl + rec_mu_sigma_loss

            if(np.isnan(loss.item())):
                print("Noped out at", epoch, j, kl, rec_comps)
                flag = True
                break

            loss.backward()
            optimizer.step()
        if(flag):
            break
        tq.set_postfix(loss=loss.item())
        #print(epoch, 'total :' + str(loss.item()) + ' rec : ' + str(rec.item()) + ' kl : ' + str(kl.sum().item()) + ' sigma: ' + str(model.log_sigma.item()))

        #break
    return model

In [5]:
model = CNN_sigmaVAE_flow()
print(model)

model = train_flow_model(model, 1500, .01, trainloader)

CNN_sigmaVAE_flow(
  (conv1): Conv1d(1, 8, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d(8, 16, kernel_size=(5,), stride=(1,))
  (conv3): Conv1d(16, 4, kernel_size=(5,), stride=(1,))
  (fc41): Linear(in_features=32, out_features=8, bias=True)
  (fc42): Linear(in_features=32, out_features=8, bias=True)
  (defc1): Linear(in_features=8, out_features=32, bias=True)
  (deconv1): ConvTranspose1d(4, 16, kernel_size=(5,), stride=(1,))
  (deconv2): ConvTranspose1d(16, 8, kernel_size=(5,), stride=(1,))
  (deconv3): ConvTranspose1d(8, 1, kernel_size=(5,), stride=(1,))
  (decoder_fc41): Linear(in_features=20, out_features=20, bias=True)
  (decoder_fc42): Linear(in_features=20, out_features=20, bias=True)
  (decoder_fc43): Linear(in_features=20, out_features=20, bias=True)
  (decoder_fc44): Linear(in_features=20, out_features=20, bias=True)
  (flow): MAF(
    (net): FlowSequential(
      (0): MADE(
        (net_input): MaskedLinear(in_features=8, out_features=50, bias=True)
        (net): Sequen

In [6]:
def evaluate_model(model, X_tensor):
    
    X_tensor = X_tensor.cuda() if torch.cuda.is_available() else X_tensor.cpu()
    X_tensor.to(device)
    out_pred, _,_,_= model(X_tensor)
    out_pred = out_pred.cpu().detach().numpy()

    idx = 0
    preds = []
    for i in range(len(out_pred)):
        for j in out_pred[i,0]:
            preds.append(j)

    preds = np.array(preds)
    
    return preds

'''
returns adjusted predictions (like DONUT method) (all others paper use this I believe also)
'''
def evaluate_anomalies(real, scores, thresh):
    
    pointwise_alerts = np.array([1 if scores[i] > thresh else 0 for i in range(len(scores))])

    anomaly_windows = []
    i = 0
    while i < len(real):
        if real[i] == 1:
            j = i
            while(j < len(real)):
                if real[j] == 0:
                    anomaly_windows.append([i,j])
                    break
                j+=1

                if j == len(real)-1 and real[j] == 1:
                    anomaly_windows.append([i,j+1])
                    break                

            i = j-1

        i+=1

    adjusted_alerts = np.copy(pointwise_alerts)
    for aw in anomaly_windows:
        if pointwise_alerts[aw[0]:aw[1]].any() == 1:
            adjusted_alerts[aw[0]:aw[1]] = 1


    return adjusted_alerts


def plot_error_and_anomaly_idxs(real, preds, anomaly_idxs):

    plt.figure(figsize=(50,15))
    plt.plot(real)
    plt.plot(preds)
    for ai in anomaly_idxs:
        plt.plot(ai, 1)
    plt.show()
    
    plt.figure(figsize=(50,15))
    scores = (preds - real[:len(preds)])**2
    for idx,ai in enumerate(anomaly_idxs):
        plt.scatter(ai, scores[ai], color='red')
    plt.plot(scores)
    plt.show()

In [7]:
real = np.zeros(len(X_test_data), dtype=np.int)
anomaly_idxs_test = anomaly_idxs - len(X_train_data)
real[anomaly_idxs_test] = 1

preds = evaluate_model(model, X_test_tensor)


In [8]:
plot_error_and_anomaly_idxs(X_test_data, preds, anomaly_idxs_test)

real = real[:len(preds)]

scores = (preds - X_test_data[:len(preds)])**2

thresh = np.quantile(scores, .99)

anomaly_preds = evaluate_anomalies(real, scores, thresh)

plt.scatter(np.arange(len(real)),real,alpha=.5)
plt.scatter(np.arange(len(real)),anomaly_preds,alpha=.5)

plt.show()



#rates:
precision = precision_score(real, anomaly_preds)
recall = recall_score(real, anomaly_preds)
f1 = f1_score(real, anomaly_preds)
print('precision : ' + str(precision) + ' recall : ' + str(recall))

precision, recall, thresholds = precision_recall_curve(real, scores)

#todo later
aupr_scores = np.copy(scores)
aupr = average_precision_score(real,scores)

print('aupr : ' + str(aupr))

plt.plot(recall, precision)
plt.show()

c:\python37\lib\site-packages\ipykernel_launcher.py:60: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
c:\python37\lib\site-packages\ipykernel_launcher.py:67: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
c:\python37\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  
c:\python37\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


precision : 0.9786223277909739 recall : 0.8
aupr : 0.26955035065195565
